# 使用keras完成数字ocr模型的训练

## 导入keras工具包

- 版本信息：

  python == 3.7

  TensorFlow == 1.13.1

  keras == 2.1.5

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Activation,Dropout,Flatten,Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

Using TensorFlow backend.


## 读入数据

考虑到原始数据较少，使用数据增强

In [2]:
# 训练集数据增强
train_datagen = ImageDataGenerator(
    rotation_range = 10,        # 随机旋转度数
    width_shift_range = 0.1,    # 随机水平平移
    height_shift_range = 0.1,   # 随机竖直平移
    rescale = 1/255,            # 数据归一化
    shear_range = 10,           # 随机错切变换
    zoom_range = 0.1,           # 随机放大
    fill_mode = 'nearest',      # 填充方式
) 

# 测试集数据增强
test_datagen = ImageDataGenerator(
    rescale = 1/255,         # 数据归一化
) 

从文件读入数据


In [3]:
# 定义 batch size
batch_size = 16

# 生成训练数据
train_generator = train_datagen.flow_from_directory(
    'ocr_data/training',
    target_size=(28,28),
    color_mode='grayscale',
    batch_size=batch_size,
    )

# 测试数据
test_generator = test_datagen.flow_from_directory(
    'ocr_data/testing',
    target_size=(28,28),
    color_mode='grayscale',
    batch_size=batch_size,
    )

Found 1409 images belonging to 10 classes.
Found 997 images belonging to 10 classes.


In [4]:
train_generator.class_indices

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

## 定义神经网络

In [5]:
# 定义顺序模型
model = Sequential()

# 第一个卷积层：output 28*28*32
# input_shape 输入平面
# filters 卷积核/滤波器个数
# kernel_size 卷积窗口大小
# strides 步长
# padding padding方式 same/valid
# activation 激活函数
model.add(Convolution2D(
    input_shape = (28,28,1),
    filters = 32,       # 卷积核的个数
    kernel_size = 5,
    strides = 1,
    padding = 'same',
    activation = 'relu'
))
# 第一个池化层：output 14*14*32
model.add(MaxPooling2D(
    pool_size = 2,
    strides = 2,
    padding = 'same',
))
# 第二个卷积层：output 14*14*64（每个卷积核对前面的32张特征图求一个新的卷积）
model.add(Convolution2D(64,5,strides=1,padding='same',activation = 'relu'))
# 第二个池化层：output 7*7*64
model.add(MaxPooling2D(2,2,'same'))
# 把第二个池化层的输出扁平化为1维：output：3316（一维向量）
model.add(Flatten())
# 第一个全连接层：output：1024（一维向量）
model.add(Dense(1024,activation = 'relu'))
# Dropout
model.add(Dropout(0.5))
# 第二个全连接层：output：10（一维向量）
model.add(Dense(10,activation='softmax'))

# 定义优化器
adam = Adam(lr=1e-4)

# 定义优化器，loss function，训练过程中计算准确率
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 训练模型并评估


由于进行了数据增强，在训练集上准确率较低，但是，会提高在测试集上的准确性，增强模型鲁棒性

In [6]:
model.fit_generator(train_generator,steps_per_epoch=len(train_generator),epochs=50,validation_data=test_generator,validation_steps=len(test_generator))

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
89/89 [==============================] - 8s 85ms/step - loss: 2.2859 - acc: 0.1320 - val_loss: 2.2177 - val_acc: 0.1374
Epoch 2/50
89/89 [==============================] - 7s 80ms/step - loss: 2.1136 - acc: 0.2704 - val_loss: 1.9911 - val_acc: 0.5426
Epoch 3/50
89/89 [==============================] - 7s 83ms/step - loss: 1.8383 - acc: 0.3554 - val_loss: 1.6435 - val_acc: 0.6369
Epoch 4/50
89/89 [==============================] - 7s 80ms/step - loss: 1.6069 - acc: 0.4404 - val_loss: 1.3754 - val_acc: 0.6499
Epoch 5/50
89/89 [==============================] - 7s 79ms/step - loss: 1.3634 - acc: 0.5344 - val_loss: 1.1396 - val_acc: 0.7442
Epoch 6/50
89/89 [==============================] - 7s 83ms/step - loss: 1.2567 - acc: 0.5765 - val_loss: 0.9329 - val_acc: 0.8666
Epoch 7/50
89/89 [==============================] - 8s 85ms/step - loss: 1.1324 - acc: 0.6236 - val_loss: 0.8787 - val_acc: 0.8325
Epoch 8/50
89/89 [=================

## 模型保存

In [8]:
model.save('number_ocr.h5')